In [15]:
# Task 3: Statistical Hypothesis Testing for Insurance Risk Drivers

# Step 1: Import Libraries
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import chi2_contingency, ttest_ind

# Step 2: Load Data
data_path = '../data/insurance_data.csv'  # adjust path if needed
df = pd.read_csv(data_path)

# Quick look
print("Data Preview:")
display(df.head())

# Step 3: Define KPI Functions
def claim_frequency(df):
    """Proportion of policies with at least one claim"""
    return (df['TotalClaims'] > 0).mean()

def claim_severity(df):
    """Average amount of a claim, given a claim occurred"""
    claims = df[df['TotalClaims'] > 0]['TotalClaims']
    return claims.mean() if not claims.empty else 0

def margin(df):
    """Profit margin = TotalPremium - TotalClaims"""
    return (df['TotalPremium'] - df['TotalClaims']).mean()

# Step 4: Hypothesis Testing Functions
def test_numeric_groups(df, group_col, metric_col):
    """T-test between two groups"""
    unique_vals = df[group_col].unique()
    if len(unique_vals) != 2:
        print(f"Skipping {group_col}: not exactly 2 groups")
        return None
    group1 = df[df[group_col] == unique_vals[0]][metric_col]
    group2 = df[df[group_col] == unique_vals[1]][metric_col]
    t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
    return p_val

def test_categorical_groups(df, group_col, target_col):
    """Chi-square test for categorical KPI"""
    contingency = pd.crosstab(df[group_col], df[target_col] > 0)
    chi2, p_val, dof, expected = chi2_contingency(contingency)
    return p_val

# Step 5: Run Hypothesis Tests
results = []

# 5a: Provinces - Risk Differences
if 'Province' in df.columns:
    provinces = df['Province'].unique()
    for a, b in combinations(provinces, 2):
        subset = df[df['Province'].isin([a, b])]
        p_val_freq = test_categorical_groups(subset, 'Province', 'TotalClaims')
        p_val_sev = test_numeric_groups(subset, 'Province', 'TotalClaims')
        results.append({
            'Hypothesis': f"Risk difference between {a} and {b}",
            'p_value_freq': p_val_freq,
            'p_value_sev': p_val_sev
        })

# 5b: Gender - Risk Differences
if 'Gender' in df.columns:
    p_val_freq = test_categorical_groups(df, 'Gender', 'TotalClaims')
    p_val_sev = test_numeric_groups(df, 'Gender', 'TotalClaims')
    results.append({
        'Hypothesis': "Risk difference between Gender",
        'p_value_freq': p_val_freq,
        'p_value_sev': p_val_sev
    })

# 5c: ZipCode - Risk Differences (first 5 codes for demo)
if 'ZipCode' in df.columns:
    zip_codes = df['ZipCode'].unique()[:5]
    for a, b in combinations(zip_codes, 2):
        subset = df[df['ZipCode'].isin([a, b])]
        p_val_freq = test_categorical_groups(subset, 'ZipCode', 'TotalClaims')
        p_val_sev = test_numeric_groups(subset, 'ZipCode', 'TotalClaims')
        results.append({
            'Hypothesis': f"Risk difference between Zip {a} and {b}",
            'p_value_freq': p_val_freq,
            'p_value_sev': p_val_sev
        })

# 5d: Margin differences between Zip Codes
if 'ZipCode' in df.columns:
    for a, b in combinations(zip_codes, 2):
        subset = df[df['ZipCode'].isin([a, b])]
        p_val_margin = test_numeric_groups(subset, 'ZipCode', 'TotalPremium') - subset['TotalClaims']
        results.append({
            'Hypothesis': f"Margin difference between Zip {a} and {b}",
            'p_value_freq': None,
            'p_value_sev': p_val_margin
        })

# Step 6: Display Results with Recommendations
results_df = pd.DataFrame(results)

def interpret_p(p):
    if p is None:
        return "Skipped"
    return "Reject H0" if p < 0.05 else "Fail to Reject H0"

results_df['Decision_Freq'] = results_df['p_value_freq'].apply(interpret_p)
results_df['Decision_Sev'] = results_df['p_value_sev'].apply(interpret_p)

print("Hypothesis Testing Results:")
display(results_df)


Data Preview:


,Age,Gender,Province,VehicleType,TotalPremium,TotalClaims,LossRatio
0,56,Male,ON,Van,1810.29,5869.37,3.24
1,69,Male,BC,Sedan,1325.00,5809.50,4.38
2,46,Male,MB,Van,1705.40,9906.79,5.81
3,32,Female,MB,SUV,1972.82,7567.30,3.84
4,60,Male,QC,Truck,790.29,4422.94,5.60


Hypothesis Testing Results:


,Hypothesis,p_value_freq,p_value_sev,Decision_Freq,Decision_Sev
0,Risk difference between ON and BC,1.0,0.054195,Fail to Reject H0,Fail to Reject H0
1,Risk difference between ON and MB,1.0,0.946279,Fail to Reject H0,Fail to Reject H0
2,Risk difference between ON and QC,1.0,0.609256,Fail to Reject H0,Fail to Reject H0
3,Risk difference between ON and AB,1.0,0.351155,Fail to Reject H0,Fail to Reject H0
4,Risk difference between BC and MB,1.0,0.057527,Fail to Reject H0,Fail to Reject H0
5,Risk difference between BC and QC,1.0,0.175255,Fail to Reject H0,Fail to Reject H0
6,Risk difference between BC and AB,1.0,0.311812,Fail to Reject H0,Fail to Reject H0
7,Risk difference between MB and QC,1.0,0.648792,Fail to Reject H0,Fail to Reject H0
8,Risk difference between MB and AB,1.0,0.376799,Fail to Reject H0,Fail to Reject H0
9,Risk difference between QC and AB,1.0,0.697167,Fail to Reject H0,Fail to Reject H0
